# CNNs
Here, we improve the Fashion-MNIST dataset to use a CNN as opposed to the neural network that I used in the last exercise.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds

tfds.disable_progress_bar()
import math
import numpy as np
import matplotlib.pyplot as plt
import logging

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


<IPython.core.display.Javascript object>

In [3]:
def normalise(images, labels):
    """
    Normalise the pixel values of a series of images to be in the [0, 1) range.
    """
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

<IPython.core.display.Javascript object>

In [13]:
dataset, metadata = tfds.load("fashion_mnist", as_supervised=True, with_info=True)
class_names = metadata.features["label"].names
train_dataset, test_dataset = dataset["train"], dataset["test"]
train_dataset = train_dataset.map(normalise).cache()
test_dataset = test_dataset.map(normalise).cache()

<IPython.core.display.Javascript object>

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
--
Total training images: 2000
Total validation images: 1000


<IPython.core.display.Javascript object>

In [5]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(
            32, (3, 3), padding="same", activation=tf.nn.relu, input_shape=(28, 28, 1)
        ),
        tf.keras.layers.MaxPooling2D((2, 2), strides=2),
        tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D((2, 2), strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(class_names)),
    ]
)
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

<IPython.core.display.Javascript object>

In [6]:
BATCH_SIZE = 32
num_train_examples = metadata.splits["train"].num_examples
num_test_examples = metadata.splits["test"].num_examples
train_dataset = (
    train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

<IPython.core.display.Javascript object>

In [8]:
model.fit(
    train_dataset, epochs=10, steps_per_epoch=math.ceil(num_train_examples / BATCH_SIZE)
)
test_loss, test_accuracy = model.evaluate(
    test_dataset, steps=math.ceil(num_test_examples / 32)
)
print("Accuracy on test dataset:", test_accuracy)

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1293 - accuracy: 0.9523
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1108 - accuracy: 0.9596
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0972 - accuracy: 0.9637
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0810 - accuracy: 0.9696
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0669 - accuracy: 0.9757
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0583 - accuracy: 0.9785
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0512 - accuracy: 0.9809
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0420 - accuracy: 0.9844
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0372 - accuracy: 0.9861
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 0.4377 

<IPython.core.display.Javascript object>